In [1]:
import sys

from pathlib import Path
import numpy as np
import pandas as pd
import scipy.signal
from scipy import signal
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm
import copy
import pickle
import lightgbm as lgb


import warnings
warnings.simplefilter("ignore")

In [2]:
class CFG:
    exp_num = 6
    n_folds = 10
    folds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    seed = 777
    local = True


In [3]:
DATA_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/ventilator-pressure-prediction")
OUTPUT_DIR = Path('./output/')
OOF_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking")
SUB_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking")
PICKLE_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_stacking_pickle")

In [4]:
sys.path.append('../../src/')
import utils as utils
from utils import Timer

In [5]:
train = pd.read_csv(DATA_DIR / "train.csv")
test = pd.read_csv(DATA_DIR / "test.csv")

In [6]:
oof_paths = sorted(list(OOF_DIR.rglob('*.npy'))+list(OOF_DIR.rglob('*.csv')))
sub_paths = sorted(list(SUB_DIR.rglob('*.npy'))+list(SUB_DIR.rglob('*.csv')))

len(oof_paths), len(sub_paths)

(29, 29)

In [7]:
with open(PICKLE_DIR / "train_only_1st_stacking.pickle", mode="rb") as f:
    oofs = pickle.load(f)

In [8]:
oofs = oofs.iloc[:, 8:].reset_index(drop=True)

In [9]:
oofs.columns = [f"pred_{i}" for i in range(len(oofs.columns))]
oofs

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12,pred_13,pred_14
0,5.833371,5.787222,6.023309,5.693756,5.820899,5.523668,5.898385,5.878293,5.883394,5.864807,5.941694,5.814224,5.805068,5.862573,5.773038
1,5.938485,5.887587,5.845187,5.917422,5.824658,5.929617,5.857195,5.873602,5.898860,5.866838,5.846353,5.847313,5.846550,5.848502,5.877314
2,8.245914,8.147405,7.962515,7.924459,8.030790,8.095352,7.991342,8.220143,8.093781,8.102347,8.084612,8.069213,7.975787,7.955888,8.036733
3,12.079426,12.057881,11.971588,12.247079,11.877783,12.134422,12.059141,12.239809,12.114367,12.041298,12.239206,12.118088,12.002528,12.016686,11.990269
4,12.523297,12.538396,12.531747,12.404102,12.507798,12.600920,12.567769,12.644754,12.594013,12.576857,12.477226,12.604493,12.502975,12.537569,12.526273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290963,29.465751,29.468914,29.481145,29.442779,29.476760,29.459700,29.461234,29.455700,29.447010,29.446421,29.463127,29.460402,29.423048,29.461472,29.469219
2290964,29.107071,29.119773,29.114014,29.128193,29.140826,29.138840,29.117303,29.121960,29.119019,29.114550,29.111472,29.111569,29.086384,29.109639,29.110482
2290965,29.874415,29.878339,29.910456,29.884028,29.887930,29.883578,29.886303,29.888802,29.877069,29.880650,29.896856,29.892732,29.874448,29.880339,29.886278
2290966,29.389308,29.391641,29.388630,29.388361,29.380993,29.397045,29.375070,29.397812,29.391010,29.390039,29.391071,29.388413,29.382864,29.373224,29.380328


In [10]:
with open(PICKLE_DIR / "test_only_1st_stacking.pickle", mode="rb") as f:
    subs = pickle.load(f)

In [11]:
subs = subs.iloc[:, 7:].reset_index(drop=True)

In [12]:
subs.columns = [f"pred_{i}" for i in range(len(subs.columns))]
subs

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9,pred_10,pred_11,pred_12,pred_13,pred_14
0,6.274139,6.234902,6.248912,6.264675,6.255715,6.270192,6.254150,6.245424,6.247398,6.248691,6.234512,6.244083,6.239510,6.234668,6.242348
1,6.000910,5.972803,5.974298,5.978864,5.977430,5.970970,5.981585,5.977894,5.978619,5.979913,5.982571,5.995547,5.973396,5.970536,5.979932
2,7.145276,7.127580,7.114281,7.185232,7.164241,7.154575,7.165225,7.141719,7.146804,7.142194,7.149453,7.138753,7.121976,7.120587,7.118664
3,7.765169,7.751527,7.740086,7.747494,7.752016,7.730515,7.737912,7.702444,7.704516,7.705604,7.772260,7.740436,7.688272,7.684845,7.707492
4,9.219263,9.211786,9.215903,9.238264,9.225886,9.231619,9.219219,9.219450,9.212603,9.214954,9.230925,9.249313,9.231130,9.221489,9.240027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023995,13.629684,0.000000,0.000000,14.043241,13.823712,13.164266,14.155994,13.728745,14.426674,28.574068,14.213044,14.063761,13.676671,14.365964,14.153107
4023996,13.785939,0.000000,0.000000,13.845679,13.816230,13.322442,14.111152,14.042326,14.669333,29.799187,14.399313,13.898353,14.014549,14.781502,14.449735
4023997,14.146443,0.000000,0.000000,13.994447,13.984568,13.530339,14.174723,14.331810,15.028529,30.255470,14.419702,14.630459,14.213983,14.877322,14.795870
4023998,14.354524,0.000000,0.000000,14.225302,14.286920,13.157774,14.750629,14.073773,14.739818,30.044895,14.306656,14.117593,14.409499,15.085758,14.273345


In [13]:
oof_paths = [Path('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_stacking_pickle/oof_lb1147.npy')]
sub_paths = [Path('/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_stacking_pickle/sub_stacking_cv0.11594.csv')]

In [14]:
def read_team_preds(paths):
    oofs = []
    for path in paths:
        path = str(path)
        if '.csv' in path:
            try:
                oof_ = pd.read_csv(path, usecols=['pressure']).values.reshape(-1)

            except:
                oof_ = pd.read_csv(path, usecols=['pred']).values.reshape(-1)
                print(type(oof_))
        else:
            try:
                oof_ = np.load(path)
            except:
                oof_ = np.load(path).reshape(-1)
#         print(f'loaded {path}')

        oofs.append(oof_)
    print('preparation done!')
    oofs = np.array(oofs)
    return oofs

In [15]:
oof_ = read_team_preds(oof_paths)
sub_ = read_team_preds(sub_paths)

preparation done!
preparation done!


In [17]:
oofs['pred_15'] = oof_[0]
subs['pred_15'] = sub_[0]

In [19]:
def get_raw_features(input_df, dataType = 'train'):
    colum = ['time_step', 'u_in', 'R', 'C']

    return input_df[colum]

In [20]:
def get_category_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    colum = ['R_C']
    rc_map = {'5_10': 0, '5_20': 1, '5_50': 2, '20_10': 3, '20_20': 4, '20_50': 5, '50_10': 6, '50_20': 7, '50_50': 8}
    
    output_df['R_C'] = [f'{r}_{c}' for r, c in zip(output_df['R'], output_df['C'])]
    output_df['R_C'] = output_df['R_C'].map(rc_map)

    return output_df[colum]

In [21]:
def get_simple_calc_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['time_delta'] = output_df.groupby('breath_id')['time_step'].diff().fillna(0)
    output_df['delta'] = output_df['time_delta'] * output_df['u_in']
    output_df['area'] = output_df.groupby('breath_id')['delta'].cumsum()

    output_df['cross']= output_df['u_in']*output_df['u_out']
    output_df['cross2']= output_df['time_step']*output_df['u_out']
    
    output_df['u_in_cumsum'] = (output_df['u_in']).groupby(output_df['breath_id']).cumsum()
    output_df['one'] = 1
    output_df['count'] = (output_df['one']).groupby(output_df['breath_id']).cumsum()
    output_df['u_in_cummean'] =output_df['u_in_cumsum'] / output_df['count']
    
    output_df['u_in_sqrt'] = output_df['u_in'].apply(lambda x: np.sqrt(x))
    output_df['u_in_sqrt_cumsum'] = output_df.groupby('breath_id')['u_in_sqrt'].cumsum()
    
    output_df = output_df.drop(['count','one'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [22]:
def get_agg_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    # Dict for aggregations
    create_feature_dict = {
        'u_in': [np.max, np.mean],
    }
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id']).agg(create_feature_dict)
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(start_time) + '_' + str(end_time))
            
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    
#     df_tmp = get_agg_window(start_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(start_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')

    output_df = pd.merge(output_df, df_agg_feature, how='left', on='breath_id')
    
    output_df['u_in_diffmax'] = output_df['u_in_amax'] - output_df['u_in']
    output_df['u_in_diffmean'] = output_df['u_in_mean'] - output_df['u_in']
    
#     output_df = output_df.drop(['u_in_amax','u_in_mean'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [23]:
def get_half_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['tmp'] = output_df['u_out']*(-1)+1 # inversion of u_out
    output_df['u_in_half'] = output_df['tmp'] * output_df['u_in']
    
#     u_in_half_max_dict = train.groupby('breath_id')['u_in_half'].max().to_dict()
#     train['u_in_half_max'] = train['breath_id'].map(u_in_half_max_dict)
#     u_in_half_min_dict = train.groupby('breath_id')['u_in_half'].min().to_dict()
#     train['u_in_half_min'] = train['breath_id'].map(u_in_half_min_dict)
    u_in_half_mean_dict = output_df.groupby('breath_id')['u_in_half'].mean().to_dict()
    output_df['u_in_half_mean'] = output_df['breath_id'].map(u_in_half_mean_dict)
#     u_in_half_std_dict = train.groupby('breath_id')['u_in_half'].std().to_dict()
#     train['u_in_half_std'] = train['breath_id'].map(u_in_half_std_dict)

    del output_df['u_in_half'], output_df['tmp']
    return output_df.iloc[:, c_num:]

In [24]:
def lowpass_filter(series, b, a):
    return signal.filtfilt(b, a, series)    

In [25]:
def get_filter_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    fp = 5 # 通過域端周波数[Hz]
    fs = 10 # 阻止域端周波数[Hz]
    gpass = 3 # 通過域端最大損失[dB]
    gstop = 40 # 阻止域端最小損失[dB]
    samplerate = 100

    fn = samplerate / 2   #ナイキスト周波数
    wp = fp / fn  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id'])['u_in'].apply(lowpass_filter, b=b, a=a)
        df_feature.name = 'u_in_filter'
                    
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    df_agg_feature = df_agg_feature.explode("u_in_filter").reset_index(drop=True)
    df_agg_feature['u_in_filter'] = df_agg_feature['u_in_filter'].astype(float)
        
    df_agg_feature['u_in_filter_cumsum'] = df_agg_feature.groupby('breath_id')['u_in_filter'].cumsum()

    return df_agg_feature.iloc[:, 1:]

In [26]:
def get_vib_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['u_out_diff'] = output_df['u_out'].diff()
    output_df['u_out_diff'].fillna(0, inplace=True)
    output_df['u_out_diff'].replace(-1, 0, inplace=True)
    uout1_df = output_df[output_df['u_out_diff']==1]
    
    first_df = output_df.loc[0::80,:]
    first_0_dict = dict(zip(first_df['id'], [0]*len(uout1_df)))

    output_df['u_in_diff'] = output_df['u_in'].diff()
    output_df['diff_sign'] = np.sign(output_df['u_in_diff'])
    output_df['sign_diff'] = output_df['diff_sign'].diff()
    output_df['tmp'] = output_df['id'].map(first_0_dict) # put 0, the 80row cycle
    output_df.iloc[0::80, output_df.columns.get_loc('sign_diff')] = output_df.iloc[0::80, output_df.columns.get_loc('tmp')]

    # Count the number of inversions, so take the absolute value and sum
    output_df['sign_diff'] = abs(output_df['sign_diff']) 
    sign_diff_dict = output_df.groupby('breath_id')['sign_diff'].sum().to_dict()
    output_df['diff_vib'] = output_df['breath_id'].map(sign_diff_dict)
    
    return output_df['sign_diff']

In [27]:
def add_time_features(out_df, input_df, dataType = 'train'):

    USE_LAG = [-2, -1, 1, 2, 3, 4]
    lag_map = {-2: 1, -1: 2, 1: 3, 2: 4, 3: 5, 4: 6}

    out_df['breath_id'] = input_df['breath_id']
    
    for lag in USE_LAG:
        out_df[f'breath_id_lag{lag_map[lag]}']=out_df['breath_id'].shift(lag).fillna(0)
        out_df[f'breath_id_lag{lag_map[lag]}same']=np.select([out_df[f'breath_id_lag{lag_map[lag]}']==out_df['breath_id']], [1], 0)

        # u_in_filter
        out_df[f'u_in_filter_lag_{lag_map[lag]}'] = out_df['u_in_filter'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_filter_diff_{lag_map[lag]}'] = out_df['u_in_filter'] - out_df[f'u_in_filter_lag_{lag_map[lag]}']
        # u_in_sqrt
        out_df[f'u_in_sqrt_lag_{lag_map[lag]}'] = out_df['u_in_sqrt'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_sqrt_diff_{lag_map[lag]}'] = out_df['u_in_sqrt'] - out_df[f'u_in_sqrt_lag_{lag_map[lag]}']

        # u_in 
        out_df[f'u_in_lag_{lag_map[lag]}'] = out_df['u_in'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_diff_{lag_map[lag]}'] = out_df['u_in'] - out_df[f'u_in_lag_{lag_map[lag]}']
        # u_out
        out_df[f'u_out_lag_{lag_map[lag]}'] = out_df['u_out'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']

        # breath_time
    out_df[f'time_step_lag_{1}'] = out_df['time_step'].shift(1).fillna(0) * out_df[f'breath_id_lag{1}same']
    out_df[f'time_step_diff_{1}'] = out_df['time_step'] - out_df[f'time_step_lag_{1}']
        
    drop_columns = ['breath_id', 'time_step_lag_1']
    drop_columns += [f'breath_id_lag{lag_map[i]}' for i in USE_LAG]
    drop_columns += [f'breath_id_lag{lag_map[i]}same' for i in USE_LAG]
    out_df = out_df.drop(drop_columns, axis=1)
    out_df = out_df.fillna(0)
    
    return out_df

In [28]:
def get_oof_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    for i in range(len(pred_cols)):
        output_df[f"pred_{i}"] = 0.
        output_df.loc[oof["u_out"] == 0, f"pred_{i}"] = _oof[f"oof{i}"].values
    

    
    return output_df['sign_diff']

In [29]:
def to_feature(input_df, dataType = 'train'):
    """input_df を特徴量行列に変換した新しいデータフレームを返す.
    """

    processors = [
        get_raw_features,
        get_simple_calc_features,
        get_agg_features,
        get_vib_features,
#         get_half_features,
        get_category_features,
        get_filter_features,
    ]

    out_df = pd.DataFrame()

    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='' + func.__name__ + ' '):
            _df = func(input_df, dataType)

        # 長さが等しいことをチェック (ずれている場合, func の実装がおかしい)
        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
#     out_df = utils.reduce_mem_usage(out_df)
#     out_df = add_time_features(out_df, input_df)
    out_df_cols = sorted(list(out_df))
    out_df = out_df[out_df_cols]
    
    return out_df

In [30]:
train_df = to_feature(train, dataType = 'train')
test_df = to_feature(test, dataType = 'test')

  0%|          | 0/6 [00:00<?, ?it/s]

get_raw_features  0.031[s]


 33%|███▎      | 2/6 [00:15<00:31,  7.90s/it]

get_simple_calc_features  15.640[s]
get_agg_features  1.027[s]


 50%|█████     | 3/6 [00:17<00:15,  5.18s/it]

...

 67%|██████▋   | 4/6 [00:12<00:04,  2.37s/it]

get_category_features  1.486[s]


 83%|████████▎ | 5/6 [00:13<00:02,  2.14s/it]

get_filter_features  7.250[s]


100%|██████████| 6/6 [00:21<00:00,  3.55s/it]


In [31]:
pred_cols = [f"pred_{i}" for i in range(len(oofs.columns))]

In [32]:
def add_oof_features(input_df, dataType = 'train'):
    
    if dataType == 'train':
        for i in range(len(pred_cols)):
            input_df[f"pred_{i}"] = 0.
            input_df.loc[train["u_out"] == 0, f"pred_{i}"] = oofs[f"pred_{i}"].values
        input_df['breath_id'] = train['breath_id']
        input_df = train_df.loc[train["u_out"] == 0].reset_index(drop=True)
    else:
        for i in range(len(pred_cols)):
            input_df[f"pred_{i}"] = 0.
            input_df.loc[:, f"pred_{i}"] = subs[f"pred_{i}"].values
        input_df['breath_id'] = test['breath_id']
        input_df = input_df.loc[test["u_out"] == 0].reset_index(drop=True)
      
    # v2
    input_df["pred_mean"] = np.mean(input_df[pred_cols].values, axis=1)
    input_df["pred_median"] = np.median(input_df[pred_cols].values, axis=1)

    input_df["pred_std"] = input_df[pred_cols].std(axis=1)
    input_df["pred_max"] = input_df[pred_cols].values.max(axis=1)
    input_df["pred_min"] = input_df[pred_cols].values.min(axis=1)
    input_df["pred_max-min"] = input_df["pred_max"] - input_df["pred_min"]
    input_df["pred_max-median"] = input_df["pred_max"] - input_df["pred_median"]
    input_df["pred_max-mean"] = input_df["pred_max"] - input_df["pred_mean"]
    input_df["pred_median-min"] = input_df["pred_median"] - input_df["pred_min"]
    input_df["pred_mean-min"] = input_df["pred_mean"] - input_df["pred_min"]
    input_df["pred_mean-median"] = input_df["pred_mean"] - input_df["pred_median"]
    input_df["pred_kurt"] = input_df[pred_cols].kurt(axis=1)
    for col_ in pred_cols:
        input_df[f"{col_}_past_1"] = input_df.groupby("breath_id")[f"{col_}"].shift(1)
        input_df[f"{col_}_past_2"] = input_df.groupby("breath_id")[f"{col_}"].shift(2)
        input_df[f"{col_}_past_3"] = input_df.groupby("breath_id")[f"{col_}"].shift(3)
        input_df[f"{col_}_past_4"] = input_df.groupby("breath_id")[f"{col_}"].shift(4)

        input_df[f"{col_}_diff_1"] = input_df[f"{col_}"] - input_df[f"{col_}_past_1"]
        input_df[f"{col_}_diff_2"] = input_df[f"{col_}"] - input_df[f"{col_}_past_2"]
        input_df[f"{col_}_diff_3"] = input_df[f"{col_}"] - input_df[f"{col_}_past_3"]
        input_df[f"{col_}_diff_4"] = input_df[f"{col_}"] - input_df[f"{col_}_past_4"]

    input_df["u_in_past_1"] = input_df.groupby("breath_id")["u_in"].shift(1)
    input_df["u_in_past_2"] = input_df.groupby("breath_id")["u_in"].shift(2)
    input_df["u_in_past_3"] = input_df.groupby("breath_id")["u_in"].shift(3)
    input_df["u_in_past_4"] = input_df.groupby("breath_id")["u_in"].shift(4)

    input_df["u_in_diff_1"] = input_df["u_in"] - input_df["u_in_past_1"]
    input_df["u_in_diff_2"] = input_df["u_in"] - input_df["u_in_past_2"]
    input_df["u_in_diff_3"] = input_df["u_in"] - input_df["u_in_past_3"]
    input_df["u_in_diff_4"] = input_df["u_in"] - input_df["u_in_past_4"]

    input_df["u_in_cumsum"] = input_df.groupby("breath_id")["u_in"].cumsum()

    del input_df['breath_id']
    
    return input_df

In [33]:
train_df = add_oof_features(train_df, dataType = 'train')
test_df = add_oof_features(test_df, dataType = 'test')

In [34]:
display(train_df), display(test_df)

,C,R,R_C,area,cross,cross2,delta,sign_diff,time_delta,time_step,...,pred_15_diff_3,pred_15_diff_4,u_in_past_1,u_in_past_2,u_in_past_3,u_in_past_4,u_in_diff_1,u_in_diff_2,u_in_diff_3,u_in_diff_4
0,50,20,5,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50,20,5,0.618632,0.0,0.0,0.618632,NaN,0.033652,0.033652,...,NaN,NaN,0.083334,NaN,NaN,NaN,18.299707,NaN,NaN,NaN
2,50,20,5,1.380843,0.0,0.0,0.762212,0.0,0.033862,0.067514,...,NaN,NaN,18.383041,0.083334,NaN,NaN,4.126236,22.425944,NaN,NaN
3,50,20,5,2.156978,0.0,0.0,0.776134,0.0,0.034028,0.101542,...,6.248865,NaN,22.509278,18.383041,0.083334,NaN,0.299544,4.425781,22.725488,NaN
4,50,20,5,3.024485,0.0,0.0,0.867507,0.0,0.034213,0.135756,...,6.655945,6.728749,22.808822,22.509278,18.383041,0.083334,2.547028,2.846573,6.972809,25.272516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290963,10,50,6,7.762919,0.0,0.0,0.062477,2.0,0.033421,0.834147,...,1.673309,1.052967,2.650333,2.438287,3.783043,3.209590,-0.780965,-0.568920,-1.913676,-1.340223
2290964,10,50,6,7.834934,0.0,0.0,0.072015,2.0,0.033427,0.867574,...,0.079825,1.331771,1.869367,2.650333,2.438287,3.783043,0.285047,-0.495918,-0.283873,-1.628629
2290965,10,50,6,7.878428,0.0,0.0,0.043494,2.0,0.033343,0.900917,...,1.112608,0.842314,2.154414,1.869367,2.650333,2.438287,-0.849980,-0.564933,-1.345899,-1.133853
2290966,10,50,6,7.936323,0.0,0.0,0.057895,2.0,0.033391,0.934309,...,-0.070026,0.621632,1.304434,2.154414,1.869367,2.650333,0.429396,-0.420585,-0.135538,-0.916503


,C,R,R_C,area,cross,cross2,delta,sign_diff,time_delta,time_step,...,pred_15_diff_3,pred_15_diff_4,u_in_past_1,u_in_past_2,u_in_past_3,u_in_past_4,u_in_diff_1,u_in_diff_2,u_in_diff_3,u_in_diff_4
0,20,5,1,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20,5,1,0.239758,0.0,0.0,0.239758,NaN,0.031904,0.031904,...,NaN,NaN,0.000000,NaN,NaN,NaN,7.515046,NaN,NaN,NaN
2,20,5,1,0.707491,0.0,0.0,0.467733,0.0,0.031924,0.063827,...,NaN,NaN,7.515046,0.000000,NaN,NaN,7.136630,14.651675,NaN,NaN
3,20,5,1,1.385252,0.0,0.0,0.677761,0.0,0.031924,0.095751,...,1.476345,NaN,14.651675,7.515046,0.000000,NaN,6.578935,13.715564,21.230610,NaN
4,20,5,1,2.224695,0.0,0.0,0.839442,0.0,0.031893,0.127644,...,3.233899,2.952690,21.230610,14.651675,7.515046,0.0,5.090346,11.669281,18.805911,26.320956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527560,10,20,3,1.969112,0.0,0.0,0.000000,0.0,0.033553,0.842145,...,0.000000,0.070302,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1527561,10,20,3,1.969112,0.0,0.0,0.000000,0.0,0.033503,0.875648,...,-0.070302,-0.070302,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1527562,10,20,3,1.973183,0.0,0.0,0.004071,1.0,0.033537,0.909185,...,-0.140604,-0.140604,0.000000,0.000000,0.000000,0.0,0.121375,0.121375,0.121375,0.121375
1527563,10,20,3,1.973183,0.0,0.0,0.000000,2.0,0.033962,0.943148,...,-0.070302,-0.070302,0.121375,0.000000,0.000000,0.0,-0.121375,0.000000,0.000000,0.000000


(None, None)

In [35]:
train_category_col = ['R_C']
train_value_col = [i for i in train_df.columns.to_list() if i not in train_category_col]

In [36]:
# norm_features = train_value_col
# train_df = train_df.fillna(0)
# test_df = test_df.fillna(0)
# scaler = RobustScaler()
# scaler.fit(train_df[norm_features])
# train_df[norm_features] = scaler.transform(train_df[norm_features].values)
# test_df[norm_features] = scaler.transform(test_df[norm_features].values)

In [37]:
y = train.loc[train["u_out"] == 0]['pressure'].reset_index(drop=True)

In [38]:
train_df = pd.concat([train_df, train.loc[train["u_out"] == 0, ['id', 'breath_id', 'pressure']].reset_index(drop=True)], axis=1)
test_df = pd.concat([test_df, test.loc[test["u_out"] == 0,['id', 'breath_id']].reset_index(drop=True)], axis=1)

In [39]:
oof_prediction = np.zeros(len(train_df))
test_preds_lst = []
input_dim = len(train_value_col)
train_df['pred'] = 0
train_gby = train_df.groupby('breath_id')['R_C'].agg('first').reset_index()
skf = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
models = []


fold_df = pd.DataFrame()
fold_df["id"] = train["id"]
fold_df["fold"] = -1

for fold, (_, valid_idx) in enumerate(skf.split(train_gby, train_gby['R_C'])):        
    valid_b_ids = train_gby.iloc[valid_idx]['breath_id'].values
    valid_df_idx = train_df[train_df['breath_id'].isin(valid_b_ids)].index.to_list()
    fold_df.loc[valid_df_idx, 'fold'] = fold

for i, fold in enumerate(range(CFG.n_folds)):
    if i not in CFG.folds:
        continue
    print(f'Fold-{fold}')
    
    train_idx = fold_df[fold_df["fold"] != fold].index
    valid_idx = fold_df[fold_df["fold"] == fold].index
    
    trn_df = train_df.loc[fold_df["fold"] != fold, train_value_col].reset_index(drop=True)
    val_df = train_df.loc[fold_df["fold"] == fold, train_value_col].reset_index(drop=True)
    trn_y = train_df.loc[fold_df["fold"] != fold, 'pressure'].reset_index(drop=True)
    val_y = train_df.loc[fold_df["fold"] == fold, 'pressure'].reset_index(drop=True)
    

    lgb_train = lgb.Dataset(trn_df, trn_y)
    lgb_valid = lgb.Dataset(val_df, val_y, reference=lgb_train)

    # LightGBM parameters
    params = {
            'boosting_type': 'gbdt',
             'objective': 'l1',
            'metric': 'l1',
            'seed': CFG.seed,
            'max_bin': 2000,
            'num_boost_round': 10000,
            'learning_rate': 0.1,
            'lambda_l1': 0.1,
            'lambda_l2': 0.1,
    }


    model = lgb.train(params, 
                      train_set=lgb_train, 
                      valid_sets=[lgb_train, lgb_valid], early_stopping_rounds=200, verbose_eval=100)
    
    models.append(model)
    
    oof_prediction[valid_idx] = model.predict(val_df[train_value_col])
    test_pred = model.predict(test_df[train_value_col])
    test_preds_lst.append(test_pred)
    score = np.abs(val_y.values - oof_prediction[valid_idx]).mean()
    print(f'fold = {fold}, score = {score}')

Fold-0
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.056333 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 358011
[LightGBM] [Info] Number of data points in the train set: 2061893, number of used features: 182
[LightGBM] [Info] Start training from score 15.820396
Training until validation scores don't improve for 200 rounds
[100]	training's l1: 0.133398	valid_1's l1: 0.13227
[200]	training's l1: 0.125604	valid_1's l1: 0.124415
[300]	training's l1: 0.122893	valid_1's l1: 0.121816
[400]	training's l1: 0.121799	valid_1's l1: 0.120997
[500]	training's l1: 0.12052	valid_1's l1: 0.120283
[600]	training's l1: 0.119691	valid_1's l1: 0.119627
[700]	training's l1: 0.119183	valid_1's l1: 0.119386
[800]	training's l1: 0.118797	valid_1's l1: 0.119102
[900]	training's l1: 0.118428	valid_1's l1: 0.118911
[1000]	training's l1: 0.118091	valid_1's l1: 0.118772
[1100]	training's l1: 0.117622	valid_1's l1: 0.1187

In [45]:
CV = np.abs(y.values - oof_prediction).mean()
CV

0.12001931707209802

In [46]:
np.save(OUTPUT_DIR / f"stacking2_oof_{CFG.exp_num}", oof_prediction)

In [47]:
unique_pressures = train["pressure"].unique()
sorted_pressures = np.sort(unique_pressures)
total_pressures_len = len(sorted_pressures)

def find_nearest(prediction):
    insert_idx = np.searchsorted(sorted_pressures, prediction)
    if insert_idx == total_pressures_len:
        # If the predicted value is bigger than the highest pressure in the train dataset,
        # return the max value.
        return sorted_pressures[-1]
    elif insert_idx == 0:
        # Same control but for the lower bound.
        return sorted_pressures[0]
    lower_val = sorted_pressures[insert_idx - 1]
    upper_val = sorted_pressures[insert_idx]
    return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val

In [43]:
oof = pd.DataFrame({'pred': oof_prediction})
oof_pp = oof['pred'].map(lambda x: unique_pressures[np.abs(unique_pressures-x).argmin()])
score = np.abs(y.values - oof_pp).mean()
print(score)

0.11843895149779683


In [44]:
sub_df = pd.read_csv(DATA_DIR / "sample_submission.csv")

sub_df.loc[test['u_out']==0, 'pressure'] = np.stack(test_preds_lst).mean(0)
sub_df.to_csv(OUTPUT_DIR / f"stacking2_submission_mean_{CFG.exp_num}.csv", index=None)

sub_df.loc[test['u_out']==0, 'pressure'] = np.median(np.stack(test_preds_lst), axis=0)
sub_df.to_csv(OUTPUT_DIR / f"stacking2_submission_median_{CFG.exp_num}.csv", index=None)

# Post Processing: https://www.kaggle.com/snnclsr/a-dummy-approach-to-improve-your-score-postprocess


sub_df = pd.read_csv(OUTPUT_DIR / f"stacking2_submission_mean_{CFG.exp_num}.csv")
sub_df.loc[test['u_out']==0, 'pressure'] = sub_df["pressure"].apply(find_nearest)
sub_df.to_csv(OUTPUT_DIR / f"stacking2_submission_mean_pp_{CFG.exp_num}.csv", index=None)

sub_df = pd.read_csv(OUTPUT_DIR / f"stacking2_submission_median_{CFG.exp_num}.csv")
sub_df.loc[test['u_out']==0, 'pressure'] = sub_df["pressure"].apply(find_nearest)
sub_df.to_csv(OUTPUT_DIR / f"stacking2_submission_median_pp_{CFG.exp_num}.csv", index=None)